## __Importing libraries__

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import geopandas as gpd

## __Setting options__

In [2]:
options = Options()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.page_load_strategy = 'eager' #to avoid the timeexception 
 

## __Establishing connection__

In [3]:
driver = webdriver.Chrome(executable_path= "C:\\selenium browser drivers\\chromedriver.exe", options= options)

driver.get("https://suchen.mobile.de/fahrzeuge/search.html?dam=0&fr=2020%3A&isSearchRequest=true&ms=17200%3B%3B12%3B%3B&s=Car&sfmr=false&vc=Car")

driver.implicitly_wait(50)
try:
    no_button = driver.find_element_by_class_name('mde-consent-accept-btn')
    no_button.click()
except:
    print('No element with class name found. skipping...')

driver.implicitly_wait(50)    
try:
    no_button1 = driver.find_element_by_xpath('//*[@id="save-search-tutorial"]/span/i')
    no_button1.click()
except:
    print('No element with class name found. skipping...')

driver.implicitly_wait(50)

C:\Users\Nikhi\AppData\Local\Temp/ipykernel_7172/94272000.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path= "C:\\selenium browser drivers\\chromedriver.exe", options= options)
C:\Users\Nikhi\AppData\Local\Temp/ipykernel_7172/94272000.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  no_button = driver.find_element_by_class_name('mde-consent-accept-btn')
C:\Users\Nikhi\AppData\Local\Temp/ipykernel_7172/94272000.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  no_button1 = driver.find_element_by_xpath('//*[@id="save-search-tutorial"]/span/i')


## __Extracting 1st Page URL links__

In [4]:
time.sleep(10)
page_sour = driver.page_source
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_sour, 'lxml')
d1 = soup.find_all('div', {'class': 'cBox cBox--content cBox--resultList'})
d2 = d1[0]
urls = []
for i in d2.find_all('a', {'class': 'link--muted no--text--decoration'}):
    urls.append(i['href'])
for i in d2.find_all('a', {'class': 'link--muted no--text--decoration result-item'}):
    urls.append(i['href'])

## __Extracting Next Page URL links__

In [5]:
def next_page(page):
    next_pages_url = f'https://suchen.mobile.de/fahrzeuge/search.html?damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&makeModelVariant1.makeId=17200&makeModelVariant1.modelGroupId=12&minFirstRegistrationDate=2020-01-01&pageNumber={page}&scopeId=C&sfmr=false'
    return next_pages_url

next_pages = []
for x in range(2,19):
    next_pages.append(next_page(x))

In [6]:
time.sleep(30)

## __Extracting URL links of all ads in all pages__

In [7]:
xurls = []
for i in next_pages:
    driver.get(i)
    ##driver.implicitly_wait(3000000)
    #time.sleep(10)
    page_sourx = driver.page_source
    soupx = BeautifulSoup(page_sourx, 'lxml')
        
    d1x = soupx.find_all('div', {'class': 'cBox cBox--content cBox--resultList'})
    d2x = d1x[0]
    for i in d2x.find_all('a', {'class': 'link--muted no--text--decoration'}):
        xurls.append(i['href'])
    for i in d2x.find_all('a', {'class': 'link--muted no--text--decoration result-item'}):
        xurls.append(i['href'])

In [8]:
urls_ = urls + xurls
len(urls_)

411

## __Extracting requried data__

In [9]:
car_names = []
gross_prices = []
net_prices = []
milages = []
vehicle_numbers = []
dealer_names = []
colors = []
dealer_locations = []
damages = []
for i in range(len(urls_)):
    
    driver.get(urls_[i])
    #time.sleep(5)
    #driver.implicitly_wait(30000000000)
    page_sour2 = driver.page_source
    soup3 = BeautifulSoup(page_sour2, 'lxml')
    
    try:
        car_name = soup3.find_all('h1', {'class': 'h2 u-margin-bottom-9 overlay-image-gallery-title'})[0].get_text()
        car_names.append(car_name)
    except IndexError:
        car_names.append('None')
    try:
        gross_price = soup3.find_all('span', {'data-testid': 'prime-price'})[0].get_text()
        gross_prices.append(gross_price)
    except IndexError:
        gross_prices.append('None')
    
    try:
        net_price = soup3.find_all('span', {'data-testid': 'sec-price'})[0].get_text()
        net_prices.append(net_price)
    except IndexError:
        net_prices.append('None')
    
    try:
        milage = soup3.find_all('div', {'class': 'g-col-6', 'id': 'mileage-v'})[0].get_text()
        milages.append(milage)
    except IndexError:
        milages.append('None')
    
    try:
        vehicle_number = soup3.find_all('div', {'class': 'g-col-6', 'id': 'sku-v'})[0].get_text()
        vehicle_numbers.append(vehicle_number)
    except IndexError:
        vehicle_numbers.append('None')
    try:
        dealer_name = soup3.find_all('a', {'id': 'dealer-hp-link-bottom'})[0].get_text()
        dealer_names.append(dealer_name)
    except IndexError:
        dealer_names.append('None')
    
    try:
        color = soup3.find_all('div', {'class': 'g-col-6', 'id': "color-v"})[0].get_text()
        colors.append(color)
    except IndexError:
        colors.append('None')
    
    try:
        dealer_location = soup3.find_all('p', {'id': 'db-address'})[0].get_text()
        dealer_locations.append(dealer_location)
    except IndexError:
        dealer_locations.append('None')
    
    try:
        damage = soup3.find_all('div', {'class': 'g-col-6', 'id': 'damageCondition-v'})[0].get_text()
        damages.append(damage)
    except IndexError:
        damages.append('None')

## __Developing a dataframe__

In [10]:
df = pd.DataFrame({'vehicle_name': car_names , 'gross_price': gross_prices, 'net_price': net_prices, 
                   'vechile_number': vehicle_numbers, 'color': colors, 'dealer_name': dealer_names,
                   'dealer_location': dealer_locations,'milage' : milages, 'damage': damages})
df.head()

,vehicle_name,gross_price,net_price,vechile_number,color,dealer_name,dealer_location,milage,damage
0,Mercedes-Benz G 63 Brabus B 800 °Starlight°24 ...,378.800 € (Brutto),318.319 € (Netto),B800-14,Schwarz,Autopoint Nordhausen GmbH,Arnoldstrasse 3DE-99734 Nordhausen,50 km,None
1,Mercedes-Benz G 63 BRABUS 700 Widestar,348.800 € (Brutto),293.109 € (Netto),G 63 Brabus- Black/Gold,Schwarz Metallic,Autopoint Nordhausen GmbH,Arnoldstrasse 3DE-99734 Nordhausen,50 km,Unfallfrei
2,"Mercedes-Benz GLS 600 Maybach 4Matic °23""°desi...",258.800 € (Brutto),217.479 € (Netto),60,Weiß Metallic,Autopoint Nordhausen GmbH,Arnoldstrasse 3DE-99734 Nordhausen,20 km,Unfallfrei
3,Mercedes-Benz S 400d 4M Brabus D40 °21 Zoll°Le...,188.800 € (Brutto),158.655 € (Netto),S400d-Box,Weiß Metallic,Autopoint Nordhausen GmbH,Arnoldstrasse 3DE-99734 Nordhausen,150 km,Unfallfrei
4,Mercedes-Benz G 500 AMG °Standheizung°Night I+...,178.800 € (Brutto),150.252 € (Netto),VL-G500,Schwarz,Autopoint Nordhausen GmbH,Arnoldstrasse 3DE-99734 Nordhausen,30 km,Unfallfrei


## __Analysis of the data__

In [11]:
# info of the data 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   vehicle_name     411 non-null    object
 1   gross_price      411 non-null    object
 2   net_price        411 non-null    object
 3   vechile_number   411 non-null    object
 4   color            411 non-null    object
 5   dealer_name      411 non-null    object
 6   dealer_location  411 non-null    object
 7   milage           411 non-null    object
 8   damage           411 non-null    object
dtypes: object(9)
memory usage: 29.0+ KB


In [12]:
#sorting the table via vehicle name
df.sort_values(by= ['vehicle_name'], inplace= True)

In [13]:
df.head(2)

,vehicle_name,gross_price,net_price,vechile_number,color,dealer_name,dealer_location,milage,damage
20,Mercedes-Benz G 350 d *AMG Sportpaket*DISTRON...,159.800 € (Brutto),134.286 € (Netto),None,Schwarz Metallic,LORD Automobile since 1999,Holderäckerstrasse 19DE-70499 Stuttgart,25.800 km,Unfallfrei
379,Mercedes-Benz G 400 d AMG *NEU* Md.2022,169.000 € (Brutto),142.017 € (Netto),None,Schwarz,Classic Cars GmbH,Lise Meitner Str. 2DE-25746 Heide/Holstein,31 km,Unfallfrei


In [14]:
#reseting the index
df.reset_index(drop = True, inplace= True)

In [15]:
df.head(2)

,vehicle_name,gross_price,net_price,vechile_number,color,dealer_name,dealer_location,milage,damage
0,Mercedes-Benz G 350 d *AMG Sportpaket*DISTRON...,159.800 € (Brutto),134.286 € (Netto),None,Schwarz Metallic,LORD Automobile since 1999,Holderäckerstrasse 19DE-70499 Stuttgart,25.800 km,Unfallfrei
1,Mercedes-Benz G 400 d AMG *NEU* Md.2022,169.000 € (Brutto),142.017 € (Netto),None,Schwarz,Classic Cars GmbH,Lise Meitner Str. 2DE-25746 Heide/Holstein,31 km,Unfallfrei


In [16]:
# info about data types
df.dtypes

vehicle_name       object
gross_price        object
net_price          object
vechile_number     object
color              object
dealer_name        object
dealer_location    object
milage             object
damage             object
dtype: object

In [17]:
#removing the 'euro' symbol and '(Brutto)'
df['gross_price'] = df['gross_price'].str.extract(r'(\d+.\d+)', expand = False)

In [18]:
#removing the 'euro' symbol and '(netto)'
df['net_price'] = df['net_price'].str.extract(r'(\d+.\d+)', expand = False)

In [19]:
#creating the space between
df['dealer_location'] = df['dealer_location'].str.replace(pat= r'DE', repl= ', DE', regex= True)

In [20]:
#creating two new columns
df[['Address','City']] = df['dealer_location'].str.split(pat = '\, DE-\d+', expand= True)

In [21]:
#checking for any duplicates
df.duplicated().sum()

64

In [22]:
#initial length
len(df)

411

In [23]:
#dropping the duplicates
df.drop_duplicates(inplace= True)

In [24]:
#final length after dropping duplicates
len(df)

347

In [25]:
#reseting the index
df.reset_index(drop = True, inplace= True)

In [26]:
#dropping the address column
df.drop('Address', axis = 1, inplace= True)

In [48]:
#dropping the last row as it has all none values
#df.drop(347, axis= 0, inplace = True)

In [27]:
#extracting the model name from the model details
df['Model'] = df['vehicle_name'].str.extract(pat = r"([^Mercedes\-Benz\s\s]\s\d+|[^Mercedes\-Benz\sG]\d+)", expand= False)

In [28]:
df.head(3)

,vehicle_name,gross_price,net_price,vechile_number,color,dealer_name,dealer_location,milage,damage,City,Model
0,Mercedes-Benz G 350 d *AMG Sportpaket*DISTRON...,159.800,134.286,None,Schwarz Metallic,LORD Automobile since 1999,"Holderäckerstrasse 19, DE-70499 Stuttgart",25.800 km,Unfallfrei,Stuttgart,G 350
1,Mercedes-Benz G 400 d AMG *NEU* Md.2022,169.000,142.017,None,Schwarz,Classic Cars GmbH,"Lise Meitner Str. 2, DE-25746 Heide/Holstein",31 km,Unfallfrei,Heide/Holstein,G 400
2,Mercedes-Benz G 400 d*G-Manufaktur/Burmester/...,174.999,NaN,None,Blau,Sinko-Automobile,"Daimlerstrasse 14, DE-73660 Urbach bei Stuttgart",7.000 km,None,Urbach bei Stuttgart,G 400


## __Generating a CSV file__

In [29]:
#df.to_csv('Cars_data.csv')

## __Further Analysis__

In [30]:
y = df['dealer_location']

In [31]:
def addr_long(x):
    from geopy.geocoders import ArcGIS
    lat_ = []
    long_ = []
    points = []
    for i in x:
        loc = ArcGIS()
        addr = loc.geocode(i, timeout=None)
        if addr is None:
            lat_.append(None)
            long_.append(None)
        else:
            longitude = addr.longitude
            latitude = addr.latitude
            points.append((latitude, longitude))
    return points
addr_long(['Seidlstraße 28, 80335 München'])

[(48.14561998850928, 11.558429986032849)]

In [32]:
plocation = [] 
for i in y[0:len(y)]:
    x = addr_long([i])
    plocation.append(x)

In [34]:
df['Location'] = plocation

In [35]:
# creating list with all pickup locations
p_location = []
for i in range(len(df['Location'])):
    x2 = df['Location'][i][0]
    p_location.append(x2)

In [36]:
df['Location'] = p_location

In [37]:
df[['lat', 'long']] = pd.DataFrame(df['Location'].tolist(), index=df.index)

In [38]:
df['Alat'] = 48.14561998850928
df['Along'] = 11.558429986032849

In [41]:
df['dealer_name'].value_counts().nlargest(6)

None                                                         32
Stern Auto GmbH - Center Leipzig                             11
AVG-Leasing & Sportwagenzentrum Vertriebs GmbH® seit 1988    11
Autopoint Nordhausen GmbH                                    11
Black Fox GmbH                                               11
Hollmann International GmbH & Co. KG                         10
Name: dealer_name, dtype: int64

In [40]:
df['dealer_location'].value_counts().nlargest(5)

Richard-Lehmann-Straße 120, DE-04277 Leipzig                  11
Münchner Straße 20, DE-82319 Starnberg am See, bei München    11
Arnoldstrasse 3, DE-99734 Nordhausen                          11
Schleißheimer Str. 222, DE-80797 München                      11
Charlotte Auerbach Str.4, DE-28816 Stuhr                      10
Name: dealer_location, dtype: int64

In [42]:
df['dealer_name'].value_counts().nsmallest(6)

Hillen Exclusive Cars     1
Miller Automobile GmbH    1
Goldberg Leasing GmbH     1
Auto 1000 GmbH            1
Autohaus ISI              1
AB luxury cars GmbH       1
Name: dealer_name, dtype: int64

In [43]:
df['dealer_location'].value_counts().nsmallest(5)

san juan de la salle, 4ES-28036 MADRID                          1
Im Steinkampe 16, DE-38110 Braunschweig                         1
Niestetalstr. 11, DE-34266 Niestetal-Heiligenrode bei Kassel    1
Fröndenberger Strasse 146, DE-58706 Menden                      1
, DE-68159 Mannheim                                             1
Name: dealer_location, dtype: int64

In [49]:
df.groupby('dealer_name')['dealer_location'].value_counts().nlargest(5)

dealer_name                                                dealer_location                                           
AVG-Leasing & Sportwagenzentrum Vertriebs GmbH® seit 1988  Münchner Straße 20, DE-82319 Starnberg am See, bei München    11
Autopoint Nordhausen GmbH                                  Arnoldstrasse 3, DE-99734 Nordhausen                          11
Black Fox GmbH                                             Schleißheimer Str. 222, DE-80797 München                      11
Stern Auto GmbH - Center Leipzig                           Richard-Lehmann-Straße 120, DE-04277 Leipzig                  11
Hollmann International GmbH & Co. KG                       Charlotte Auerbach Str.4, DE-28816 Stuhr                      10
Name: dealer_location, dtype: int64

In [51]:
df.groupby('dealer_name')['dealer_location'].value_counts().nsmallest()

dealer_name          dealer_location                      
A. Kahn Design Ltd.  Kings Road 385GB-SW10 0LX London         1
AB luxury cars GmbH  Kurfürstendamm 40/41, DE-10719 Berlin    1
ACE Cars Exclusive   Heerlerbaan 76NL-6418CH Heerlen          1
AL Ain Class Motors  Dubai  U.A.E ..AE-1711 Dubai             1
AL SAHAB GERMANY     Karlstraße 79, DE-72766 Reutlingen       1
Name: dealer_location, dtype: int64

In [55]:
def addr_long(x):
    from geopy.geocoders import ArcGIS
    lat_ = []
    long_ = []
    points = []
    for i in x:
        loc = ArcGIS()
        addr = loc.geocode(i, timeout=None)
        if addr is None:
            lat_.append(None)
            long_.append(None)
        else:
            longitude = addr.longitude
            latitude = addr.latitude
            points.append((latitude, longitude))
    return points
addr_long(['Charlotte Auerbach Str.4, DE-28816 Stuhr'])

[(52.997479989657876, 8.775680041301854)]

In [56]:
d = { 'Pick_locat' : [[48.0005900133404, 11.351039992343175],[51.49731998401569, 10.790259992140477],[48.16903998990034, 11.565229972622006], [51.314110020787496, 12.398530036396608], [52.997479989657876, 8.775680041301854]],
        'Address' : ['AVG-Leasing & Sportwagenzentrum Vertriebs GmbH', 'Autopoint Nordhausen GmbH', 'Black Fox GmbH', 'Stern Auto GmbH - Center Leipzig', 'Hollmann International GmbH & Co. KG']}
df1 = pd.DataFrame(d)

In [57]:
# Top locations
# # 48.14562,	-11.55843 germany location
import folium
center = [48.14562,	-11.55843]
map_germany = folium.Map(location=center, zoom_start=5)

for i,r in df1.iterrows():
    folium.Marker(location=r['Pick_locat'],
                  popup = r['Address'],
                  tooltip='Location').add_to(map_germany)
#display map
map_germany

In [63]:
d2 = { 'locat' : [[51.48335613160815, -0.17692343321200876],[52.50216998742064, 13.321410013978095],[50.870253085564, 5.995634182354252], [25.269510000000025, 55.30884000000003], [48.50163998478169, 9.219229989321548]],
        'Address' : ['A. Kahn Design Ltd', 'AB luxury cars GmbH', 'ACE Cars Exclusive', 'AL Ain Class Motors', 'AL SAHAB GERMANY']}
df2 = pd.DataFrame(d2)

In [64]:
# least locations
# # 48.14562,	-11.55843 germany location
import folium
center = [48.14562,	-11.55843]
map_germany = folium.Map(location=center, zoom_start=5)

for i,r in df2.iterrows():
    folium.Marker(location=r['locat'],
                  popup = r['Address'],
                  tooltip='Location').add_to(map_germany)
#display map
map_germany